# TLDR LLM solutions for ~not reading~ software manuals 

#### Anton Antonov   
#### RakuForPrediction at WordPress   
#### RakuForPrediction-book at GitHub
#### August 2023

------

## Introduction

In this notebook we use [Large Language Model (LLM) functions](https://rakuforprediction.wordpress.com/2023/08/01/workflows-with-llm-functions/), [AAp1, AA1], for generating (hopefully) executable, correct, and harmless code for Operating System resources managements.

In order to be concrete and useful, we take the Markdown files of the articles ["It's time to rak!"](https://dev.to/lizmat/series/20329), that explain the motivation and usage of the Raku module ["App::Rak"](https://raku.land/zef:lizmat/App::Rak), and we show how meaningful, file finding shell commands can be generated via LLMs exposed to the code-with-comments from those articles.

In other words, we prefer to apply the attitude Too Long; Didn't Read (TLDR) to articles and related Raku module [help](https://github.com/lizmat/App-Rak/blob/main/README.md).
(Because "App::Rak" is useful, but it has too many parameters that we prefer not learn that much about.)  

**Remark:** We say that "App::Rak" uses a Domain Specific Language (DSL), which is done with Raku's Command Line Interface (CLI) features.

### Procedure outline

1. Clone the corresponding [article repository](https://github.com/lizmat/articles)
2. Locate and ingest the "App::Rak" dedicated Markdown files
3. Extract code blocks from the Markdown files
   - Using ["Markdown::Grammar"](https://raku.land/zef:antononcube/Markdown::Grammar) functions
4. Get comment-and-code line pairs from the code blocks
   - Using Raku text manipulation capabilities
      - (After observing code examples) 
5. Generate from the comment-and-code pairs LLM few-shot training rules
6. Use the LLM example function to translate natural language commands into (valid and relevant) "App::Rak" DSL commands
   - With a few or a dozen natural language commands 
7. Use LLMs to generate natural language commands in order to test LLM-TLDR-er further 

Step 6 says how we do our TLDR -- we use LLM-translations of natural language commands.

### Alternative procedure

Instead of using Raku to process text we can make LLM functions for extracting the comment-and-code pairs.
(That is also shown below.)    

### Article's structure

The exposition below follows the outlines of procedure subsections above.

The article/document/notebook was made with the Jupyter framework, using the Raku package ["Jupyter::Kernel"](https://raku.land/cpan:BDUGGAN/Jupyter::Kernel), [BD1].  

## Setup

In [3]:
use Markdown::Grammar;
use Data::Reshapers;
use Data::Summarizers;
use LLM::Functions;
use Text::SubParsers;

## File names

In [24]:
my $dirName = $*HOME ~ '/GitHub/lizmat/articles';
my @fileNames = dir($dirName).grep(*.Str.contains('time-to-rak'));
@fileNames.elems

4

In [26]:
my %texts = @fileNames.map({ $_.basename => slurp($_) });
%texts.elems

4

In [28]:
%texts>>.chars

{its-time-to-rak-1.md => 7437, its-time-to-rak-2.md => 8725, its-time-to-rak-3.md => 14181, its-time-to-rak-4.md => 9290}

With the function `md-section-tree` we extract code blocks from Markdown documentation files into data structures amenable for further programmatic manipulation (in Raku.)

In [32]:
my %docTrees = %texts.map({ $_.key => md-section-tree($_.value, modifier => 'Code', max-level => 0) });
%docTrees>>.elems

{its-time-to-rak-1.md => 1, its-time-to-rak-2.md => 11, its-time-to-rak-3.md => 24, its-time-to-rak-4.md => 16}

In [34]:
my @sections = %docTrees.values.Array.&flatten

[```
# Search for literal string "foo" from the current directory
$ rak foo

# Search for "foo" in the "lib" directory
$ rak foo lib

# Search for "foo" only in files with .txt and .text extensions
$ rak foo --extensions=txt,text

# Find files that have "lib" in their name from the current dir
$ rak lib --find

# Show all unique "name" fields in JSON files
$ rak --json-per-file '*<name>' --unique

# Show all lines with numbers between 1 and 65
$ rak '/ \d+ <?{ 1 <= $/.Int <= 65 }> /'

# Show number of files / lines authored by Scooby Doo
$ rak --blame-per-line '*.author eq "Scooby Doo"' --count-only
``` ```
# produce results without any highlighting
$ rak foo --/highlight

# produce results as if piping to a file
$ rak foo --no-human
``` ```
# specify a literal pattern at the end of a line
$ rak --type=ends-with foo

# same, with a regular expression
$ rak '/ foo $/'
``` ```
# save --ignorecase as -i, without description
$ rak --ignorecase --save=i

# save --ignoremark as -m, with desc

In [36]:
my @rules;
@sections.map({ 
    given $_ { 
        for m:g/ '#' $<comment>=(\V+) \n '$' $<code>=(\V+) \n / -> $m {
           @rules.push( ($m<comment>.Str.trim => $m<code>.Str.trim) ) 
         } } })

(Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil Nil)

Here is the number of rules:

In [38]:
@rules.elems

69

Here is a sample of the rules:

In [40]:
.say for @rules.pick(6)

Show all lines starting with "t" titlecased => rak '{.tc if /^ t /}' twenty
Only accept Raku and Markdown files => rak foo --extensions=#raku,#markdown
save --ignoremark as -m, with description => rak --ignoremark --description='Ignore any accents' --save=m
Look for lines with either "foo" or "bar" => rak '/ one | four /' twenty
Convert all occurrences of "teen" into "10" => rak '.subst("teen",10)' twenty --type=code
Reverse the order of the characters of each line => rak '*.flip' twenty


In [42]:
to-pretty-table(@rules, align => 'l', field-names => <Key Value>)

+---------------------------------------------------------------------------+---------------------------------------------------------------------+
| Key                                                                       | Value                                                               |
+---------------------------------------------------------------------------+---------------------------------------------------------------------+
| Search for literal string "foo" from the current directory                | rak foo                                                             |
| Search for "foo" in the "lib" directory                                   | rak foo lib                                                         |
| Search for "foo" only in files with .txt and .text extensions             | rak foo --extensions=txt,text                                       |
| Find files that have "lib" in their name from the current dir             | rak lib --find                    

### Code generation examples

Here we define an LLM function for generating "App::Rak" shell commands:

In [52]:
my &frak = llm-example-function(@rules, e => llm-evaluator('PaLM'))

-> **@args, *%args { #`(Block|5060278236888) ... }

In [56]:
my @cmds = ['Find files that have ".nb" in their names', 'Find files that have ".nb"  or ".wl" in their names',
 'Show all directories of the parent directory', 'Give me files without extensions and that contain the phrase "notebook"', 
 'Show all that have extension raku or rakumod and contain Data::Reshapers'];

my @tbl = @cmds.map({ %( 'Command' => $_, 'App::Rak' => &frak($_) ) }).Array;

@tbl.&dimensions

(5 2)

Here is a table showing the natural language commands and  

In [58]:
to-pretty-table(@tbl, align=>'l', field-names => <Command App::Rak>)

+--------------------------------------------------------------------------+--------------------------------------------------------------------+
| Command                                                                  | App::Rak                                                           |
+--------------------------------------------------------------------------+--------------------------------------------------------------------+
| Find files that have ".nb" in their names                                | rak --find --extensions=nb                                         |
| Find files that have ".nb"  or ".wl" in their names                      | rak --find --extensions=nb,wl                                      |
| Show all directories of the parent directory                             | rak --find --parent --/file                                        |
| Give me files without extensions and that contain the phrase "notebook"  | rak --extensions= --type=contains notebook     

-------

## Translating random generated commands

Consider testing the applicability of the approach by generating a "good enough" sample of natural language commands for finding files or directories.

We can generate such commands via LLM. Here we define an LLM function with two parameters the returns a Raku list:

In [66]:
my &fcg = llm-function({"Generate $^_a natural language commands for finding $^b in a file system. Give the commands as a JSON list."}, form => sub-parser('JSON'))

-> **@args, *%args { #`(Block|5060312614472) ... }

In [80]:
my @gCmds1 = &fcg(4, 'files').flat;
@gCmds1.raku

["Find me all the files with the word 'report' in their title", "Show me any files with the extension '.doc'", "Display all the photos in the 'Pictures' folder", "Show me all the files that were modified today"]

Here are the corresponding translations to the "App::Rak" DSL:

In [82]:
my @tbl1 = @gCmds.map({ %( 'Command' => $_, 'App::Rak' => &frak($_) ) }).Array;
to-pretty-table(@tbl1, align=>'l', field-names => <Command App::Rak>)

+------------------------------------------+--------------------------------------+
| Command                                  | App::Rak                             |
+------------------------------------------+--------------------------------------+
| Show me all the files inside this folder | rak --find .                         |
| Where are the files located?             | current directory and subdirectories |
| What files are inside this directory?    | rak --find                           |
| List all the files in this folder        | rak --find                           |
+------------------------------------------+--------------------------------------+

Let use redo the generation and translation using different specs: 

In [78]:
my @gCmds2 = &fcg(4, 'files that have certain extensions or contain certain words').flat;
@gCmds2.raku

["Find me all .pdf files", "Show me all .docx documents", "Locate all files with 'project' in the title", "Display files with the extension .xlsx"]

In [86]:
my @tbl2 = @gCmds2.map({ %( 'Command' => $_, 'App::Rak' => &frak($_) ) }).Array;
to-pretty-table(@tbl2, align=>'l', field-names => <Command App::Rak>)

+----------------------------------------------+----------------------------+
| Command                                      | App::Rak                   |
+----------------------------------------------+----------------------------+
| Find me all .pdf files                       | rak --extensions=pdf       |
| Show me all .docx documents                  | rak --types=docx           |
| Locate all files with 'project' in the title | rak --title=project --find |
| Display files with the extension .xlsx       | rak --extensions=xlsx      |
+----------------------------------------------+----------------------------+

-------

## Alternative programming with LLM

In [20]:
my &fcex = llm-function({"Extract the consecutive line pairs in which the first start with '#' and second with '\$' from the text $_. Group them as a key-value pairs and put them JSON format."}) 

-> **@args, *%args { #`(Block|5060278153024) ... }

In [21]:
&fcex(@sections[0])

Variable '@sections' is not declared.  Perhaps you forgot a 'sub' if
this was intended to be part of a signature?

()

------

## References

### Articles

[AA1] Anton Antonov, ["Workflows with LLM functions"](https://rakuforprediction.wordpress.com/2023/08/01/workflows-with-llm-functions/), (2023), [RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages, paclets

[AAp1] Anton Antonov, [LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions), (2023), [GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov, [WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI), (2023), [GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov, [WWW::PaLM Raku package](https://github.com/antononcube/Raku-WWW-PaLM), (2023), [GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov, [Text::SubParsers Raku package](https://github.com/antononcube/Raku-Text-SubParsers), (2023), [GitHub/antononcube](https://github.com/antononcube).

[BD1] Brian Duggan, [Jupyter::Kernel Raku package](https://raku.land/cpan:BDUGGAN/Jupyter::Kernel), (2017-2023), [GitHub/bduggan](https://github.com/bduggan/raku-jupyter-kernel).